In [4]:
import pandas as pd
from sqlalchemy.dialects.mssql.information_schema import columns

df = pd.read_csv('extractedMimic.csv')  # Replace with the path to your CSV file


In [5]:
df.head()

,icustay_id,hadm_id,intime,outtime,dbsource,suspected_infection_time_poe,suspected_infection_time_poe_days,specimen_poe,positiveculture_poe,antibiotic_time_poe,...,glucose_min1,glucose_max1,glucose_mean,rrt,subject_id,hadm_id.1,icustay_id.1,urineoutput,colloid_bolus,crystalloid_bolus
0,205941,156324,28/5/2157 14:26:21,30/5/2157 14:18:24,metavision,28/5/2157 15:30:00,-0.044201,MRSA SCREEN,0,28/5/2157 00:00:00,...,40.0,202.0,87.250000,0,88883,156324,205941,0.0,NaN,250.0
1,252848,163315,29/7/2196 02:26:17,29/7/2196 12:02:39,metavision,29/7/2196 04:57:00,-0.104664,MRSA SCREEN,1,29/7/2196 00:00:00,...,182.0,231.0,206.500000,0,46154,163315,252848,0.0,NaN,250.0
2,237901,180937,14/2/2145 17:55:07,23/2/2145 12:43:43,metavision,14/2/2145 21:20:00,-0.142280,BLOOD CULTURE,0,15/2/2145 00:00:00,...,123.0,185.0,151.285714,1,42682,180937,237901,0.0,NaN,250.0
3,207491,143962,11/6/2159 12:47:02,14/6/2159 16:31:30,metavision,11/6/2159 12:11:00,0.025023,BLOOD CULTURE,0,11/6/2159 00:00:00,...,92.0,118.0,105.000000,0,45111,143962,207491,4.0,NaN,250.0
4,293063,118489,1/1/2135 17:28:33,2/1/2135 06:56:56,metavision,1/1/2135 15:55:00,0.064965,BLOOD CULTURE,0,1/1/2135 00:00:00,...,150.0,163.0,155.000000,0,56648,118489,293063,5.0,NaN,250.0


In [46]:
from sklearn.model_selection import train_test_split

# Step 1: Prepare the dataset for machine learning
# Drop the ICUSTAY_ID and ROW_ID columns, and separate features from labels
X = df[['urineoutput', 'lactate_min','Bun_mean'.lower(),'Sysbp_min'.lower(), 'Metastatic_cancer'.lower(), 'Inr_max'.lower(), 'Age'.lower(), 'Sodium_max'.lower(), 'Aniongap_max'.lower(), 'Creatinine_min'.lower(), 'Spo2_mean'.lower()]]
# 'Urineoutput'.lower(), 'Lactate_min'.lower(), 'Bun_mean'.lower(), 'Sysbp_min'.lower(), 'Metastatic_cancer'.lower(), 'Inr_max'.lower(), 'Age'.lower(), 'Sodium_max'.lower(), 'Aniongap_max'.lower(), 'Creatinine_min'.lower(), 'Spo2_mean'.lower()
y = df['thirtyday_expire_flag']

# Step 2: Handle missing values (example: filling with median or mean)
# This can be adjusted based on your specific needs and data distribution
X.fillna(X.median(), inplace=True)  # Fill missing values with median

# Step 3: Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print shapes of the resulting datasets
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (3647, 11)
Testing set shape: (912, 11)


C:\Users\beta\AppData\Local\Temp\ipykernel_7388\2835009340.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.median(), inplace=True)  # Fill missing values with median


In [47]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 1: Train the XGBoost model
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Step 2: Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Step 3: Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print evaluation metrics
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)


Model Accuracy: 83.11%
Confusion Matrix:
[[690  50]
 [104  68]]
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.93      0.90       740
           1       0.58      0.40      0.47       172

    accuracy                           0.83       912
   macro avg       0.72      0.66      0.68       912
weighted avg       0.81      0.83      0.82       912

